In [1]:
#necessary libraries
import re
import chardet 
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud

from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D

from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('/content/full_eng.csv')

data

Unnamed: 0                                               Text  label
0              0                                          Next part      0
1              1                 Iii8mllllllm\nMdxfvb8o90lplppi0005      0
2              2  🤣🤣😂😂🤣🤣🤣😂osm vedio ....keep it up...make more v...      0
3              3  What the fuck was this? I respect shwetabh and...      0
4              4  Concerned authorities should bring arundathi R...      0
...          ...                                                ...    ...
8167         516  @ Itch Pong Abe Randi ki Aulad, what has happe...      2
8168         517  Arundhati your janma diya maap gyan ji hi to m...      2
8169         518                                             mutiny      2
8170         519                               Kung Thu Gadhiya Hai      2
8171         520  Friend brotherYou are cheating on me with logs...      2

[8172 rows x 3 columns]

In [3]:
data_test = pd.read_csv('/content/test.csv')

In [4]:
data_test.head()

ID  ... Sub-task B
0    C7.2589  ...       NGEN
1    C68.872  ...       NGEN
2    C36.762  ...        GEN
3  C4.1540.1  ...       NGEN
4     C59.68  ...       NGEN

[5 rows x 4 columns]

In [ ]:
#data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [6]:
NAG = data_test[(data_test['Sub-task A']=='NAG')]
OAG = data_test[(data_test['Sub-task A']=='CAG')]
CAG = data_test[(data_test['Sub-task A']=='OAG')]

In [7]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG.head()

Text  Sub-task A
0        U deserve more subscribers. U really great.           0
1                                     Nice video....           0
2  sorry if i bother somebody.. iam a defence asp...           0
3  Joker was amazing....it was not glamorised !.....           0
4                                          Nice baro           0

In [8]:
data_test = pd.concat([NAG,
                  OAG,
                  CAG
                 ],axis=0)

In [9]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
#from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
#import transformers

In [10]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=0cf1b18cb74b01f07760479ecf1640b960293692312441e41bfeb6b03850a58f
  Stored in directory: /tmp/pip-ephem-wheel-cache-xc7k4m1f/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [11]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [12]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [13]:
def get_vec(x):
  doc = nlp(str(x))
  vec = doc.vector
  return vec

In [14]:
data['vec'] = data['Text'].apply(lambda x: get_vec(x))

In [15]:
data_test['vec'] = data_test['Text'].apply(lambda x: get_vec(x))

In [16]:
X = data['vec'].to_numpy()
X = X.reshape(-1, 1)

In [17]:
P = data_test['vec'].to_numpy()
P = P.reshape(-1, 1)

In [18]:
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

In [19]:
P = np.concatenate(np.concatenate(P, axis = 0), axis = 0).reshape(-1, 300)

In [20]:
y = data['label']

In [21]:
Q = data_test['Sub-task A']

In [22]:
X_train = X
X_test = P
y_train = y 
y_test = Q



In [23]:
clf = LogisticRegression(solver='liblinear')

In [24]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
y_pred = clf.predict(X_test)

In [26]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.82      0.84       836
           1       0.23      0.24      0.24       117
           2       0.31      0.38      0.34       113

    accuracy                           0.71      1066
   macro avg       0.47      0.48      0.47      1066
weighted avg       0.73      0.71      0.72      1066

